In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import skimage.io
from skimage.io import imread, imsave
import cv2
import pickle
#from skimage.metrics import structural_similarity as ssim

In [2]:
path = "/home/groups/ChangLab/dharani/OHSU-TMA/"

# plot1 
#img1 = imread(os.path.join(path, "Contours/BR1506-A015 - Scene 002.png"))
#plt.imshow(img1[1000:1500, 1000:1500])
#plt.title("Contours")
#plt.show()

# plot 2
#img2 = imread(os.path.join(path, "Foreground/BR1506-A015 - Scene 002.png"))
#plt.imshow(img2[1000:1500, 1000:1500])
#plt.title("Foreground")
#plt.show()

# plot 3
img3 = imread(os.path.join(path, "Labels/BR1506-A015 - Scene 059.tif"))
#plt.imshow(img3[1000:1500, 1000:1500])
#plt.title("Labels")
#plt.show()

# plot 4
#img4 = imread(os.path.join(path, "Originals/BR1506-A015 - Scene 002.png"))
#plt.imshow(img4[1000:1500, 1000:1500])
#plt.title("Originals")
#plt.show()


In [3]:
# plot 5
img5 = imread(os.path.join(path, "Segmentations/Mesmer/BR1506-A015 - Scene 059.tif"))
#plt.imshow(img5[1000:1500, 1000:1500])
#plt.title("Mesmer")
#plt.show()

# plot 6
#img6 = imread(os.path.join(path, "Segmentations/Stardist/dsb/BR1506-A015 - Scene 003.png"))
#plt.imshow(img6[1000:1500, 1000:1500])
#plt.title("stardist-dsb")
#plt.show()

# plot 7
img7 = imread(os.path.join(path, "Segmentations/Stardist/BR1506-A015 - Scene 059.png"))
#plt.imshow(img7[1000:1500, 1000:1500])
#plt.title("stardist-vf")
#plt.show()

# plot 8
img8 = imread(os.path.join(path, "Segmentations/Cellpose/BR1506-A015 - Scene 059.png_mask.png"))
#plt.imshow(img8[1000:1500, 1000:1500])
#plt.title("Cellpose")
#plt.show()

# plot 9
img9 = imread(os.path.join(path, "Segmentations/UnMicst/BR1506-A015 - Scene 059.tif"))
#plt.imshow(img9[1000:1500, 1000:1500])
#plt.title("UnMicst")
#plt.show()

### DICE Coefficient

In [4]:
gd_img = img3
#gd_img_uint8 = np.uint8(gd_img) 
gd_img_binary = (gd_img > 0).astype(int)

pred_img = img7 * 0.25
#pred_img_uint8 = np.uint8(pred_img) 
pred_img_binary = (pred_img > 0).astype(int)

In [5]:
#print(ssim(gd_img_binary, pred_img_binary))

def dice_coeff(true, pred):
    true_f = true#.flatten()
    pred_f = pred#.flatten()
    union = np.sum(true_f) + np.sum(pred_f)
    if union==0:
        return 1
    intersection = np.sum(true_f * pred_f)
    return (2.* intersection/union)

dice_score = dice_coeff(gd_img_binary, pred_img_binary)
print(dice_score)

0.8491044918130658


### IOU 

In [7]:
def jaccard_idx(true, pred):
    intersection = (true * pred).sum()
    union = true.sum() + pred.sum() #- intersection
    return intersection / union
    
iou_score = jaccard_idx(gd_img_binary, pred_img_binary)
print(iou_score)

0.4245522459065329


### PMasks - DICE and IOU

In [ ]:
path = "/home/groups/ChangLab/dharani/OHSU-TMA/"
path_pmasks = os.path.join(path, "results_mscx/pmasks")
files = sorted(os.listdir(path_pmasks))

with open(os.path.join(path_pmasks, files[0]), 'rb') as f:
    values = pickle.load(f)

In [ ]:
for keys, value in values.items():
    if keys == 4:
        for methods, array in value.items():
            if methods == 'mean':
                print(array)
                
                dice_score = dice_coeff(gd_img_uint8, array, k=255)
                print(dice_score)

In [ ]:
values[4]["Mesmer"].astype(int)

### Pixel wise DICE & IOU calculation

In [ ]:
gd_img = img3
gd_img_binary = (gd_img > 0).astype(int)

mesmer = img5
mesmer_img_binary = (mesmer > 0).astype(int)

stardist = img7
stardist_img_binary = (stardist > 0).astype(int)

cellpose = img8
cellpose_img_binary = (cellpose > 0).astype(int)

unmicst = img9
unmicst_img_binary = (unmicst > 0).astype(int)

In [ ]:
# different weights 

print("Weights of 12")

#        "Mesmer":   {4:0.21, 8:0.24, 12:0.38, 16:0.46, 20:0.40, 24:0.27, 28:0.18, 32:0.15}, 
#        "Stardist": {4:0.26, 8:0.28, 12:0.20, 16:0.11, 20:0.07, 24:0.06, 28:0.06, 32:0.07}, 
#        "Cellpose":{4:0.23, 8:0.24, 12:0.27, 16:0.26, 20:0.34, 24:0.37, 28:0.43, 32:0.58}, 
#        "UnMicst": {4:0.28, 8:0.23, 12:0.13, 16:0.15, 20:0.16, 24:0.28, 28:0.31, 32:0.18}

# equal weights
#total_sum =  mesmer_img_binary * 0.25 + stardist_img_binary * 0.25 + cellpose_img_binary * 0.25 + unmicst_img_binary * 0.25#all
#total_sum =  mesmer_img_binary * 0.25 + stardist_img_binary * 0.25 + cellpose_img_binary * 0.25  # w/o unmicst
#total_sum =  mesmer_img_binary * 0.25 + stardist_img_binary * 0.25 + unmicst_img_binary * 0.25 # w/o cellpose
#total_sum =  mesmer_img_binary * 0.25 + cellpose_img_binary * 0.25 + unmicst_img_binary * 0.25 # w/o stardist
#total_sum =  stardist_img_binary * 0.25 + cellpose_img_binary * 0.25 + unmicst_img_binary * 0.25 # w/o mesmer

# unequal weights
#total_sum =  mesmer_img_binary * 0.38 + stardist_img_binary * 0.20 + cellpose_img_binary * 0.27 + unmicst_img_binary * 0.13 #all
#total_sum =  mesmer_img_binary * 0.38 + stardist_img_binary * 0.20 + cellpose_img_binary * 0.27  # w/o unmicst
#total_sum =  mesmer_img_binary * 0.38 + stardist_img_binary * 0.20 + unmicst_img_binary * 0.13 # w/o cellpose
#total_sum =  mesmer_img_binary * 0.38 + cellpose_img_binary * 0.27 + unmicst_img_binary * 0.13 # w/o stardist
total_sum =  stardist_img_binary * 0.20 + cellpose_img_binary * 0.27 + unmicst_img_binary * 0.13 # w/o mesmer

print(np.unique(total_sum))

In [ ]:
total_img_binary = (total_sum > 0.5).astype(int)
print(np.max(total_img_binary))
print(np.unique(total_img_binary))

In [ ]:
dice_score = dice_coeff2(gd_img_binary, total_img_binary)
print(dice_score)

iou_score = jaccard_idx(gd_img_binary, total_img_binary)
print(iou_score)